In [2]:
import pandas as pd

from pendulum import now

In [3]:
dataset_name = 'sidra_tab3939_dejetos'
raiz = '../databases'

### Bronze

In [4]:
contador = 1
ano_inicial = 1990

ultimo_ano = now().year - 1
for ano in range(ano_inicial, ultimo_ano):
    if contador == 1:
        df_3939 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/3939/n6/all/v/all/p/{ano}/c79/32793,32794,32796')
        contador += 1
    else:
        df_3939 = pd.concat(
            [df_3939,
             (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/3939/n6/all/v/all/p/{ano}/c79/32793,32794,32796')
              .iloc[1:])],
            ignore_index=True
            )
        contador += 1

df_3939.to_json(f'{raiz}/bronze_{dataset_name}.json')

### Prata

|  CÓDIGO |        CULTURA        |
|---------|-----------------------|
| 32793   | Galináceos - galinhas |
| 327930  | Galináceos - frangos  |
| 32794   |  Suínos - total       |
| 32796   | Galináceos - total    |

In [5]:
df = (pd.read_json(f'{raiz}/bronze_{dataset_name}.json')
 .drop(0, axis = 0)
 .filter(['D1C', 'D3C', 'D4C', 'V'])
 .query('D4C in ("32793", "32794", "32796")')
 .reset_index(drop=True)
 .assign(**{
     'V': lambda df: pd.to_numeric(df['V'], errors='coerce')
     })
    )

df_frangos = (pd.DataFrame()
             .assign(**{
                 'D1C': df.iloc[1::3, 0].reset_index(drop=True),
                 'D3C': df.iloc[1::3, 1].reset_index(drop=True),
                 'D4C': '327930',
                 'V': df.iloc[2::3, 3].reset_index(drop=True) - df.iloc[::3, 3].reset_index(drop=True)
             })
            )

(pd.concat(
    [df.query('D4C != "32796"'),
     df_frangos]
    )
 .assign(**{
        'uf': lambda df: df['D1C'].str.slice(0,2).astype(int),
        'kg_cabeca': lambda df: pd.to_numeric(df['V'], errors='coerce'),
        'municipio': lambda df: df['D1C'].astype(int),
        'ano': lambda df: df['D3C'].astype(int),
        'cultura': lambda df: df['D4C'].astype(int),
        'fonte': 'Dejetos'
        })
 .dropna(subset='kg_cabeca')
 .convert_dtypes()
 .groupby(['uf', 'municipio', 'ano', 'cultura', 'fonte'])
 .agg({'kg_cabeca':sum})
 .to_parquet(f'{raiz}/prata_{dataset_name}.parquet')
 )

/tmp/ipykernel_15116/687714621.py:35: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'kg_cabeca':sum})


### Ouro

Cálculos:

N: ((kg_cabeça * Teor N)/1000) * ((100 - perdas_N)/100)

P, K: (kg_cabeca * Teor N, K)/1000

In [6]:
parametro = {
    32793: {'n': 0.58, 'p': 0.18, 'k': 0.21}, # de Galinhas
    327930: {'n': 0.4, 'p': 0.12, 'k': 0.24}, # de Frangos
    32794: {'n': 14.2, 'p': 2.3, 'k': 6.2}, # Suínos
}

# Perdas por volatilização do N
perdas_N_galinaceos = 0
perdas_N_suinos = 0

In [7]:
df = (pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet')
 .groupby(['uf', 'ano', 'cultura', 'fonte'])
 .agg({'kg_cabeca':sum})
 .reset_index()
 .assign(
      N = lambda df: df.apply(lambda lin: ((lin['kg_cabeca'] * parametro[lin['cultura']]['n'])/1000) * ((100 - perdas_N_suinos)/100) 
          if lin['cultura'] == 32794 else
          ((lin['kg_cabeca'] * parametro[lin['cultura']]['n'])/1000) * ((100 - perdas_N_galinaceos)/100), axis = 1),
      P= lambda df: df.apply(lambda lin: (lin['kg_cabeca'] * parametro[lin['cultura']]['p'])/1000, axis = 1),
      K= lambda df: df.apply(lambda lin: (lin['kg_cabeca'] * parametro[lin['cultura']]['k'])/1000, axis = 1),
      fonte = lambda df: df['cultura'].apply(lambda x: 'Dejetos suínos' if x == 32794 else 'Dejetos galináceos')
  )
 .drop('kg_cabeca', axis = 1)
 .melt(id_vars=['uf', 'ano', 'cultura', 'fonte'],
       value_vars=['N', 'P', 'K'],
       var_name='elemento',
       value_name='valor')
 .convert_dtypes()
 .groupby(['uf', 'ano', 'fonte', 'elemento'])
 .agg({'valor':sum})
 .sort_index()
 )

galinha_df = df.query('fonte == "Dejetos galináceos"').squeeze()
galinha_df.to_csv(f'{raiz}/ouro_npk_dejetos_galinaceos.csv')

suino_df = df.query('fonte == "Dejetos suínos"').squeeze()
suino_df.to_csv(f'{raiz}/ouro_npk_dejetos_suinos.csv')

/tmp/ipykernel_15116/245983958.py:3: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'kg_cabeca':sum})
/tmp/ipykernel_15116/245983958.py:20: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'valor':sum})


In [8]:
galinha_df

uf  ano   fonte               elemento
11  1990  Dejetos galináceos  K           1279.06473
                              N           2385.13818
                              P            722.15094
    1991  Dejetos galináceos  K           1470.24231
                              N            2751.1967
                                             ...    
53  2022  Dejetos galináceos  N           3457.08388
                              P             1049.457
    2023  Dejetos galináceos  K           2081.96439
                              N           3837.72238
                              P           1160.91102
Name: valor, Length: 2721, dtype: Float64

In [9]:
suino_df

uf  ano   fonte           elemento
11  1990  Dejetos suínos  K            5288.6744
                          N           12112.7704
                          P            1961.9276
    1991  Dejetos suínos  K             6238.378
                          N            14287.898
                                         ...    
53  2022  Dejetos suínos  N             2456.103
                          P             397.8195
    2023  Dejetos suínos  K             864.0692
                          N            1978.9972
                          P             320.5418
Name: valor, Length: 2748, dtype: Float64